<a href="https://colab.research.google.com/github/OEAJR77/Dinesh/blob/main/cncnetwrk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.datasets.cifar10 import load_data

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# Load and preprocess data
(x_train, y_train), (x_test, y_test) = load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
# Build a simple CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 30s 47ms/step - loss: 1.5846 - accuracy: 0.4277 - val_loss: 1.3371 - val_accuracy: 0.5247
Epoch 2/10
625/625 [==============================] - 28s 44ms/step - loss: 1.2469 - accuracy: 0.5600 - val_loss: 1.2158 - val_accuracy: 0.5710
Epoch 3/10
625/625 [==============================] - 28s 46ms/step - loss: 1.1069 - accuracy: 0.6120 - val_loss: 1.0822 - val_accuracy: 0.6214
Epoch 4/10
625/625 [==============================] - 28s 44ms/step - loss: 1.0184 - accuracy: 0.6426 - val_loss: 1.0703 - val_accuracy: 0.6261
Epoch 5/10
625/625 [==============================] - 28s 44ms/step - loss: 0.9500 - accuracy: 0.6685 - val_loss: 1.0204 - val_accuracy: 0.6408
Epoch 6/10
625/625 [==============================] - 28s 46ms/step - loss: 0.9060 - accuracy: 0.6852 - val_loss: 1.0121 - val_accuracy: 0.6554
Epoch 7/10
625/625 [==============================] - 28s 45ms/step - loss: 0.8603 - accuracy: 0.7020 - val_loss: 0.9442 - val_accuracy:

In [ ]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


313/313 [==============================] - 3s 8ms/step - loss: 0.9474 - accuracy: 0.6816
Test accuracy: 0.6815999746322632


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping
import time


In [ ]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [ ]:
def create_and_train_cnn(num_layers, num_units, epochs, learning_rate):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))

    for _ in range(num_layers):
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    for _ in range(num_units):
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))

    model.add(Dense(10, activation='softmax'))

    # Compile the model with customized learning rate
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy', AUC()])

    # Train the model
    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=epochs, validation_split=0.2, callbacks=[EarlyStopping(patience=3)])
    end_time = time.time()
    training_time = end_time - start_time

    return history, training_time


In [ ]:
# Define the hyperparameter search space
num_layers_list = [2, 3]
num_units_list = [64, 128]
epochs_list = [20, 30]
learning_rate_list = [0.001, 0.01]

best_model = None
best_val_accuracy = 0

for num_layers in num_layers_list:
    for num_units in num_units_list:
        for epochs in epochs_list:
            for learning_rate in learning_rate_list:
                print(f"Training model with num_layers={num_layers}, num_units={num_units}, epochs={epochs}, learning_rate={learning_rate}")
                history, training_time = create_and_train_cnn(num_layers, num_units, epochs, learning_rate)

                # Evaluate the model on the validation set
                val_accuracy = history.history['val_accuracy'][-1]
                print(f"Validation accuracy: {val_accuracy:.4f}, Training time: {training_time:.2f} seconds\n")

                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    best_model = history.model


Training model with num_layers=2, num_units=64, epochs=20, learning_rate=0.001
Epoch 1/20
1250/1250 [==============================] - 130s 95ms/step - loss: 2.3032 - accuracy: 0.1004 - auc: 0.5013 - val_loss: 2.3029 - val_accuracy: 0.0977 - val_auc: 0.5007
Epoch 2/20
1250/1250 [==============================] - 115s 92ms/step - loss: 2.3031 - accuracy: 0.0968 - auc: 0.4982 - val_loss: 2.3028 - val_accuracy: 0.0952 - val_auc: 0.4971
Epoch 3/20
1250/1250 [==============================] - 114s 91ms/step - loss: 2.3028 - accuracy: 0.0970 - auc: 0.4992 - val_loss: 2.3028 - val_accuracy: 0.0977 - val_auc: 0.4970
Epoch 4/20
1250/1250 [==============================] - 117s 93ms/step - loss: 2.3029 - accuracy: 0.0985 - auc: 0.4975 - val_loss: 2.3029 - val_accuracy: 0.0952 - val_auc: 0.4971
Epoch 5/20
1250/1250 [==============================] - 115s 92ms/step - loss: 2.3028 - accuracy: 0.1006 - auc: 0.4987 - val_loss: 2.3026 - val_accuracy: 0.1016 - val_auc: 0.4996
Epoch 6/20
1250/1250 [====

In [4]:
test_loss, test_accuracy, test_auc = best_model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test AUC: {test_auc:.4f}")


AttributeError: ignored

In [5]:
from tensorflow.keras.models import Sequential

# Initialize best_model with a dummy model
best_model = Sequential()

best_val_accuracy = 0

for num_layers in num_layers_list:
    for num_units in num_units_list:
        for epochs in epochs_list:
            for learning_rate in learning_rate_list:
                print(f"Training model with num_layers={num_layers}, num_units={num_units}, epochs={epochs}, learning_rate={learning_rate}")
                history, training_time = create_and_train_cnn(num_layers, num_units, epochs, learning_rate)

                # Evaluate the model on the validation set
                val_accuracy = history.history['val_accuracy'][-1]
                print(f"Validation accuracy: {val_accuracy:.4f}, Training time: {training_time:.2f} seconds\n")

                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    best_model = model  # Update best_model with the current model


Training model with num_layers=2, num_units=64, epochs=20, learning_rate=0.001


NameError: ignored

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping
import time

# Define the create_and_train_cnn function
def create_and_train_cnn(num_layers, num_units, epochs, learning_rate):
    # ... (Rest of the function code)

# Load and preprocess the data
# ... (Load and preprocess data as previously mentioned)

# Initialize best_model with a dummy model
best_model = Sequential()

best_val_accuracy = 0

# Define the hyperparameter search space
num_layers_list = [2, 3]
num_units_list = [64, 128]
epochs_list = [20, 30]
learning_rate_list = [0.001, 0.01]

for num_layers in num_layers_list:
    for num_units in num_units_list:
        for epochs in epochs_list:
            for learning_rate in learning_rate_list:
                print(f"Training model with num_layers={num_layers}, num_units={num_units}, epochs={epochs}, learning_rate={learning_rate}")
                history, training_time = create_and_train_cnn(num_layers, num_units, epochs, learning_rate)

                # Evaluate the model on the validation set
                val_accuracy = history.history['val_accuracy'][-1]
                print(f"Validation accuracy: {val_accuracy:.4f}, Training time: {training_time:.2f} seconds\n")

                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    best_model = model  # Update best_model with the current model


IndentationError: ignored

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping
import time

# Define the create_and_train_cnn function
def create_and_train_cnn(num_layers, num_units, epochs, learning_rate):
    # ... (Rest of the function code)

# Load and preprocess the data
# ... (Load and preprocess data as previously mentioned)

# Initialize best_model with a dummy model
best_model = Sequential()

best_val_accuracy = 0

# Define the hyperparameter search space
num_layers_list = [2, 3]
num_units_list = [64, 128]
epochs_list = [20, 30]
learning_rate_list = [0.001, 0.01]

for num_layers in num_layers_list:
    for num_units in num_units_list:
        for epochs in epochs_list:
            for learning_rate in learning_rate_list:
                print(f"Training model with num_layers={num_layers}, num_units={num_units}, epochs={epochs}, learning_rate={learning_rate}")
                history, training_time = create_and_train_cnn(num_layers, num_units, epochs, learning_rate)

                # Evaluate the model on the validation set
                val_accuracy = history.history['val_accuracy'][-1]
                print(f"Validation accuracy: {val_accuracy:.4f}, Training time: {training_time:.2f} seconds\n")

                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    best_model = model  # Update best_model with the current model


IndentationError: ignored